<a href="https://colab.research.google.com/github/wuchenyu38/18ma573chenyuwu/blob/master/Vasicek_Calibration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Vasicek model verification:
$$
dr_t=\kappa(\mu-r_t)dt+\sigma dW_t
\\
dr_t+r_tdt=\kappa\mu dt+\sigma dW_t
\\
e^{kt}-r_0=\mu (e^{kt}-1)+\int_0^t e^{ks}dW_s
\\
r_t=r_0e^{-\kappa t}+\mu(1-e^{-\kappa t})+\sigma e^{-\kappa t}\int_0^t e^{\kappa S}dW_s
$$

In [0]:
import numpy as np
import scipy.stats as ss
import scipy.optimize as so
import matplotlib.pyplot as plt

In [0]:
def price(T, kappa, sigma, mu, r0):
  B = (1 - np.exp(-kappa *(T-0)))/kappa
  A = (mu-sigma**2/(2*kappa**2))*(B-(T-0))-(sigma**2/(4*kappa))*B**2
  return np.exp(A-B*r0)

In [0]:
def sample(T, kappa, sigma, mu, r0, num_path):
  mu_r = (mu*T) + ((r0 - mu)*(1-np.exp(-1*kappa*T))/kappa)
  var_r = ((sigma**2)/(2*(kappa**3)))*((2*kappa*T) - 3 + (4*np.exp(-1*kappa*T)) - np.exp(-2*kappa*T))
  r = np.random.normal(mu_r,var_r,num_path)
  r_output = np.exp(-1 * r)
  return r_output.mean()

In [0]:
theta = [0.1, 0.05, 0.003, 0.03]
kappa, mu, sigma, r0 = theta
num_path=100

In [0]:
fp = price(1,kappa,sigma,mu,r0)
sp = sample(1,kappa,sigma,mu,r0,num_path)

print("ZCB P(0,1) price by ZCB formula is " + str(fp))
print("ZCB P(0,1) price by exact sampling is " + str(sp))

ZCB P(0,1) price by ZCB formula is 0.9695084475425054
ZCB P(0,1) price by exact sampling is 0.9695061741431313


In [0]:
def Libor(T, P):
  lib=(100/T)*((1/P)-1)
  return lib

In [0]:
print("LIBOR L(0,1) price using ZCB formula price is " + str(Libor(1,fp)))
print("LIBOR L(0,1) price using ZCB sample price is " + str(Libor(1,sp)))

LIBOR L(0,1) price using ZCB formula price is 3.145052787810565
LIBOR L(0,1) price using ZCB sample price is 3.1452946531072667


In [0]:
def swap_formula(T,N,kappa,sigma,mu,r0):
  delta=T/N
  fs=np.zeros(N)
  for i in range(N):
    fs[i] = price(T,kappa, sigma,mu,r0)
  
  result=100*(1-fs[N-1])/(delta*sum(fs))
  return result

In [0]:
def swap_sample(T,N,kappa,sigma,mu,r0,num_path):
  delta=T/N
  ss=np.zeros(N)
  for i in range(N):
    ss[i] = sample(T, kappa, sigma, mu, r0, num_path)
  
  result=100*(1-ss[N-1])/(delta*sum(ss))
  return result

In [0]:
s_explicit = swap_formula(5,10,kappa,sigma,mu,r0)
s_sample = swap_sample(5,10,kappa,sigma,mu,r0,num_path)

print("10-term swap rate with term length 1/2 year using the formula is " + str(s_explicit))
print("10-term swap rate with term length 1/2 year using exact sampling is " + str(s_sample))


10-term swap rate with term length 1/2 year using the formula is 3.733970072840246
10-term swap rate with term length 1/2 year using exact sampling is 3.7371949855118567


In [0]:
import pandas as pd
dfLiborRate = pd.DataFrame({'maturity (months)': [1, 2, 3, 6, 12],
                           '20081029 rate(%)': [3.1175, 3.2738, 3.4200, 3.4275, 3.4213],
                           '20110214 rate(%)': [0.2647, 0.2890, 0.3140, 0.4657, 0.7975]
                           })

libor_2008_10_29 = [3.1175, 3.2738, 3.4200, 3.4275, 3.4213];
libor_2011_02_14 = [0.2647, 0.2890, 0.3140, 0.4657, 0.7975];
libor_maturities = [1/12, 2/12, 3/12, 6/12, 12/12]

In [0]:
def vasicek_libor_error_function(theta, market_rates, maturities):
  err1 = 0
  kappa, mu, sigma, r0 = theta
  
  for i in np.arange(len(market_rates)):
    P = price(maturities[i],kappa,mu,sigma,r0)
    
    calc_libor =Libor(maturities[i],P)
    
    err1 = err1 + ((calc_libor - market_rates[i])/market_rates[i])**2
  return err1

def libor_calibration(libor_rates,maturities):
  # Supply initial guess. Use theta array from assignment as baseline.
  init_theta = np.array([0.1, 0.05, 0.003, 0.03])
  init_kappa, init_mu, init_sigma, init_r0 = init_theta
  return so.fmin(vasicek_libor_error_function, init_theta, args = (libor_rates, maturities), disp = 0)

In [0]:
calibrated_theta = libor_calibration(libor_2008_10_29,libor_maturities)

print(">>>Calibrated kappa is " + str(calibrated_theta[0]))
print(">>>Calibrated mu is " + str(calibrated_theta[1]))
print(">>>Calibrated sigma is " + str(calibrated_theta[2]))
print(">>>Calibrated r_0 is " + str(calibrated_theta[3]))

>>>Calibrated kappa is -0.16751822866885324
>>>Calibrated mu is 0.2272785401828623
>>>Calibrated sigma is -0.11172465115200159
>>>Calibrated r_0 is 0.030702484976398742
